In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from datetime import datetime as dt
import glob
import numpy as np
import os
import sys

sys.path.append('./Tools_V1/')

from tools_V1.general import SNNToolset
from tools_V1.model import TNUtilsModelBlocks
from tools_V1.inferencelite import ToolsInferenceLite
from tools_V1.similarities_self_2D import PlotSimilaritiesSelf2D

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [16]:
tools_general = SNNToolset()
tools_model = TNUtilsModelBlocks()
tools_inference_lite = ToolsInferenceLite()
tools_plot_2d = PlotSimilaritiesSelf2D()

### 1. Source paths

In [17]:
modelId="model2"
epochTrained="dir_dst_model_epochs_20"
organismName="Stickleback"

path_dataset_json = "/media/hmorales/Skynet/TwinNet/"+organismName+"/embryos_test.json"
embryos_normal = sorted(tools_general.fn_json_load(path_dataset_json)['normal_bright_complete'])

In [18]:
dir_dst_2D = "/media/hmorales/Skynet/TwinNet/"+organismName+"/output/"+modelId+"_"+epochTrained+"/"

### 2. Embedding model

In [19]:
path_model_old = "/media/hmorales/Skynet/TwinNet/"+organismName+"/models/"+modelId+"/"+epochTrained+"/"

In [20]:
resnet50 = tools_model.snn_embedding_resnet50_make()

2.11.0


In [21]:
resnet50.load_weights(path_model_old)

### 3. Loop through paths: Calculate embeddings and similarities, save similarities

In [22]:
exp_imgs_embs_similarities = dict()

In [23]:
time_start = dt.now()
for i in range(len(embryos_normal)):
    embryo_normal = embryos_normal[i]
    duration = dt.now() - time_start
    print(f"[{str(i+1).zfill(4)}/{len(embryos_normal)}][{duration}]".ljust(50),
          end='\n')
    
    exp_emb_id = tools_inference_lite.fn_stem_name_experiment_embryo(f"{embryo_normal}/*.tif")
    imgs, embs, sims = tools_inference_lite.dir_embryo_to_similarities_self(embryo_normal, resnet50)
    exp_imgs_embs_similarities[exp_emb_id] = {
        'paths_imgs': imgs,
        'array_embs': embs,
        'list_similarities': sims
    }
    #print(type(sims))
    print(len(sims))
    #for imagepath in imgs:
    #    print(imagepath)

[0001/7][0:00:00.000086]                          
1349E] Image embeddings 27/27                     
[0002/7][0:00:13.494977]                          
1349E] Image embeddings 27/27                     
[0003/7][0:00:27.047126]                          
1349E] Image embeddings 27/27                     
[0004/7][0:00:40.637388]                          
1349E] Image embeddings 27/27                     
[0005/7][0:00:54.161523]                          
1349E] Image embeddings 27/27                     
[0006/7][0:01:07.640730]                          
1349E] Image embeddings 27/27                     
[0007/7][0:01:21.192488]                          
1349E] Image embeddings 27/27                     


### Save image paths, embeddings, similarities to files

In [24]:
for _k_embryo, (_v_k_imgs, _v_k_embs, _v_k_sims) in exp_imgs_embs_similarities.items():
    print(_k_embryo)
    # Embryo directory for data
    path_dst_dir_data = tools_general.fn_dirs_make(f'{dir_dst_2D}/data/{_k_embryo}/')
    
    # Image paths
    tools_general.fn_json_write({'paths_imgs': exp_imgs_embs_similarities[_k_embryo][_v_k_imgs]},
                                f"{path_dst_dir_data}/20221022_{_k_embryo}_2Dplot_paths_images.json")
    
    # Embeddings
    tools_general.fn_npy_write(exp_imgs_embs_similarities[_k_embryo][_v_k_embs],
                               f"{path_dst_dir_data}/20221022_{_k_embryo}_2Dplot_embeddings.npy")
    # Similarities
    tools_inference_lite.fn_similarities_save(path_dst_dir_data,
                                              exp_imgs_embs_similarities[_k_embryo][_v_k_sims],
                                              embryo_id=_k_embryo)

210709_Normal--XY10--E001
210709_Normal--XY10--E002
210709_Normal--XY15--E001
210709_Normal--XY15--E002
210709_Normal--XY20--E001
210709_Normal--XY20--E002
210709_Normal--XY20--E003


### Load image paths, embeddings, similarities to files

In [25]:
for i in range(len(embryos_normal)):
    embryo_normal = embryos_normal[i]
    exp_emb_id = tools_inference_lite.fn_stem_name_experiment_embryo(f"{embryo_normal}/*.tif")
    path_dst_dir_data = f"{dir_dst_2D}/data/{exp_emb_id}/"

    exp_imgs_embs_similarities[exp_emb_id] = {
        'paths_imgs': tools_general.fn_json_load(glob.glob(
            f"{path_dst_dir_data}*_2Dplot_paths_images.json")[0])['paths_imgs'],
        'array_embs': np.load(glob.glob(f"{path_dst_dir_data}*_2Dplot_embeddings.npy")[0]),
        'list_similarities': tools_inference_lite.fn_similarities_load(path_dst_dir_data)
    }
    #print(path_dst_dir_data)
    #temp = tools_inference_lite.fn_similarities_load(path_dst_dir_data)
    #length = len(temp)
    #print(temp[50])
    #for i in range(length):
    #    print(temp[i])

### 4. Plot similarities 2D

### E000

In [26]:
dir_dst_plots=dir_dst_2D
for _k_embryo, _v_embryo in exp_imgs_embs_similarities.items():

    # Embryo directory for plots
    path_dst_dir_data = tools_general.fn_dirs_make(f'{dir_dst_plots}/plots/{_k_embryo}/')
    print(path_dst_dir_data)    
    # Plot 2D similarities
    path_save_fig = "{}/{}_sims_autoregression.svg" \
        .format(path_dst_dir_data, _k_embryo)
    path_save_GridZ = "{}/{}_sims_autoregression_gridZ.mat" \
        .format(path_dst_dir_data, _k_embryo)     
    path_save_GridCV = "{}/{}_sims_autoregression_gridZ.csv" \
        .format(path_dst_dir_data, _k_embryo)      
    tools_plot_2d.plot_embryo_similarities_self_2d(
        _v_embryo['list_similarities'],
        _v_embryo['paths_imgs'],
        color_adjust=True,
        lamb=0.1,
        path_saveCV=path_save_GridCV,
        path_saveZ=path_save_GridZ,
        path_save=path_save_fig)

/media/hmorales/Skynet/TwinNet/Stickleback/output/model2_dir_dst_model_epochs_20//plots/210709_Normal--XY10--E001/
1350
True
(1349, 1349)
(1349, 1349)
/media/hmorales/Skynet/TwinNet/Stickleback/output/model2_dir_dst_model_epochs_20//plots/210709_Normal--XY10--E002/
1350
True
(1349, 1349)
(1349, 1349)
/media/hmorales/Skynet/TwinNet/Stickleback/output/model2_dir_dst_model_epochs_20//plots/210709_Normal--XY15--E001/
1350
True
(1349, 1349)
(1349, 1349)
/media/hmorales/Skynet/TwinNet/Stickleback/output/model2_dir_dst_model_epochs_20//plots/210709_Normal--XY15--E002/
1350
True
(1349, 1349)
(1349, 1349)
/media/hmorales/Skynet/TwinNet/Stickleback/output/model2_dir_dst_model_epochs_20//plots/210709_Normal--XY20--E001/
1350
True
(1349, 1349)
(1349, 1349)
/media/hmorales/Skynet/TwinNet/Stickleback/output/model2_dir_dst_model_epochs_20//plots/210709_Normal--XY20--E002/
1350
True
(1349, 1349)
(1349, 1349)
/media/hmorales/Skynet/TwinNet/Stickleback/output/model2_dir_dst_model_epochs_20//plots/210709